In [63]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import os
import state_code
from state_code import SAMPLE
import collections
import plotly.graph_objects as go
import numpy as np

In [64]:
source = requests.get('https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports/')

soup = bs(source.text,'html.parser')
page = soup.prettify()

In [66]:
web_fileset = set()
web_set = set()
dic_p = {}
for i in soup.find_all('a', class_='js-navigation-open'):
    if 'csv' in i['href']:
        web_set.add(i['href'])
        web_fileset.add(i['href'].split('/')[-1])
        dic_p[i['href'].split('/')[-1]] = i['href']

dir_set = set(os.listdir('./dataset'))
new_data = web_fileset.difference(dir_set)
for i in new_data:
    path = 'https://raw.githubusercontent.com'+''.join(dic_p[i].split('/blob'))
    # raw_source = requests.get(path)
    # raw_soup = bs(raw_source.text, 'html.parser')
    # csv_path = 'https://raw.githubusercontent.com'+raw_soup.find(id='raw-url')['href']
    print(path)
    df = pd.read_csv(path)
    filename = i.split('/')[-1]
    #print(filename)
    df.to_csv('./dataset/'+filename)


https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/02-28-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-20-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-22-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-20-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-11-2020.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports